![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Finance/7.2.Entity_Resolution_with_NASDAQ.ipynb)

# Financial Entity Resolution

**In this notebook, we continue from where left off in `7.Entity_Resolution` notebook.**

# Installation

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from johnsnowlabs import nlp, finance
nlp.install(force_browser=True)

# Start Spark Session

In [ ]:
spark = nlp.start()

👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


# Sentence Entity Resolver Models

Entity resolution is an important task in natural language processing and information extraction, as it allows for more accurate analysis and understanding of financial texts. For example, in a news article discussing the performance of a company's stock, accurately identifying and disambiguating the company's name is crucial for accurately tracking the stock's performance.

An NLP use case in financial or legal applications is identifying financial entities' presence in a given text. One of those entities could be `Company Name`. We can carry out NER to extract different chunks of information, but in real financial and legal use cases, the company name is usually not useful as it is mentioned in the text. Sometimes we need the _official_ name of the company (instead of `Amazon`, `Amazon.com INC`, as registered in Edgar, Crunchbase and Nasdaq). We have pre-trained sentence entity resolver models for these purposes shown below with the examples.

# Pretrained Entity Resolution Models for Finance

Here are the list of pretrained Entity Resolution models:


|index|model|
|-----:|:-----|
| 1| [Company Name Normalization Using Nasdaq](https://nlp.johnsnowlabs.com/2022/10/22/finel_nasdaq_data_company_name_en.html)  |
| 2| [Company Name Normalization Using Edgar Database](https://nlp.johnsnowlabs.com/2022/08/30/finel_edgar_company_name_en.html)  |
| 3| [Company Names Normalization Using Crunchbase](https://nlp.johnsnowlabs.com/2022/09/28/finre_work_experience_en.html)  | 
| 4| [Company Name to Ticker Using Nasdaq](https://nlp.johnsnowlabs.com/2022/10/22/finel_nasdaq_data_ticker_en.html)  | 
| 5| [Company Name to IRS Number Using Edgar Database](https://nlp.johnsnowlabs.com/2022/08/30/finel_edgar_irs_en.html)  |
| 6| [Resolve Tickers to Company Names Using Nasdaq](https://nlp.johnsnowlabs.com/2022/09/09/finel_tickers2names_en.html)  |
| 7| [Resolve Company Names to Tickers Using Nasdaq](https://nlp.johnsnowlabs.com/2022/09/08/finel_names2tickers_en.html)  | 

## Common Componennts

Other than providing the code in the "result" field it provides more metadata about the matching process:

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- all_k_resolutions -> All codes descriptions
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId

We will use following Generic Function For Getting the Codes and Relation Pairs

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

def get_codes (lp, text, vocab='company_name', hcc=False):

    """Returns LightPipeline resolution results"""
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for i in range(len(full_light_result)):

      for chunk, code in zip(full_light_result[i]['ner_chunk'], full_light_result[i][vocab]):   
          begin.append(chunk.begin)
          end.append(chunk.end)
          chunks.append(chunk.result)
          codes.append(code.result) 
          all_codes.append(code.metadata['all_k_results'].split(':::'))
          resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
          all_distances.append(code.metadata['all_k_distances'].split(':::'))
          all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
          if hcc:
              try:
                  all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
              except:
                  all_k_aux_labels.append([])
          else:
              all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes, 'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    return df

## Company Name Normalization using NASDAQ




[Nasdaq Homepage](https://www.nasdaq.com/)

`The Nasdaq Stock Market` (National Association of Securities Dealers Automated Quotations Stock Market) is an American stock exchange based in New York City. It is ranked second on the list of stock exchanges by market capitalization of shares traded, behind the New York Stock Exchange

Let's suppose we get this text from scrapping the Internet, or from Twitter. Firstly, we get company name from sample text with `finner_orgs_prods_alias` model.

In [ ]:
text = "Aspect Development provides component and supplier management (CSM) technology to improve the product-development cycle of manufacturing."

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

ner_model = finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")
    
ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = nlp.Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings,
        ner_model,
        ner_converter,
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

lp_ner = nlp.LightPipeline(model)

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[OK!]
finner_orgs_prods_alias download started this may take some time.
[OK!]


In [ ]:
ner_result = lp_ner.annotate(text)

ner_result


{'document': ['Aspect Development provides component and supplier management (CSM) technology to improve the product-development cycle of manufacturing.'],
 'ner_chunk': ['Aspect Development'],
 'token': ['Aspect',
  'Development',
  'provides',
  'component',
  'and',
  'supplier',
  'management',
  '(',
  'CSM',
  ')',
  'technology',
  'to',
  'improve',
  'the',
  'product-development',
  'cycle',
  'of',
  'manufacturing',
  '.'],
 'ner': ['B-ORG',
  'I-ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'embeddings': ['Aspect',
  'Development',
  'provides',
  'component',
  'and',
  'supplier',
  'management',
  '(',
  'CSM',
  ')',
  'technology',
  'to',
  'improve',
  'the',
  'product-development',
  'cycle',
  'of',
  'manufacturing',
  '.'],
 'sentence': ['Aspect Development provides component and supplier management (CSM) technology to improve the product-development cycle of manufacturing.']}

In [ ]:
ner_result["ner_chunk"]

['Aspect Development']

Alright! We extract company name from sample text. Now, we normalize `Aspect Development` company using `finel_nasdaq_data_company_name` model.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use_lg", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_nasdaq_data_company_name", "en", "finance/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("name")\
      .setDistanceFunction("EUCLIDIAN")

pipeline = nlp.Pipeline(
      stages = [
          document_assembler,
          embeddings,
          resolver])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

lp = nlp.LightPipeline(model)

tfhub_use_lg download started this may take some time.
Approximate size to download 753.3 MB
[OK!]
finel_nasdaq_data_company_name download started this may take some time.
[OK!]


In [ ]:
ner_result["ner_chunk"]

['Aspect Development']

In [ ]:
%time
get_codes (lp, ner_result["ner_chunk"], vocab='name')

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 12.4 µs


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Aspect Development,0,17,ASPECT DEVELOPMENT INC,"[ASPECT DEVELOPMENT INC, BINDVIEW DEVELOPMENT CORP, COHESION TECHNOLOGIES INC, ENVISION DEVELOPMENT CORP, ASPECT COMMUNICATIONS CORP, CONCEPTS DIRECT INC, CATELLUS DEVELOPMENT CORP, APPLIED GRAPHICS TECHNOLOGIES INC, MOVING IMAGE TECHNOLOGIES INC, APPROACH RESOURCES INC, ENTORIAN TECHNOLOGIES INC, BIOAFFINITY TECHNOLOGIES INC, ANALYSIS & TECHNOLOGY INC, BOTTOMLINE TECHNOLOGIES INC, IMAGE GUIDED TECHNOLOGIES INC, CREATIVE TECHNOLOGY LTD, EZGO TECHNOLOGIES LTD, EXCO RESOURCES INC, COMVERSE TECHNOLOGY INC, ADVANCED ANALOGIC TECHNOLOGIES INC]","[ASPECT DEVELOPMENT INC, BINDVIEW DEVELOPMENT CORP, COHESION TECHNOLOGIES INC, ENVISION DEVELOPMENT CORP, ASPECT COMMUNICATIONS CORP, CONCEPTS DIRECT INC, CATELLUS DEVELOPMENT CORP, APPLIED GRAPHICS TECHNOLOGIES INC, MOVING IMAGE TECHNOLOGIES INC, APPROACH RESOURCES INC, ENTORIAN TECHNOLOGIES INC, BIOAFFINITY TECHNOLOGIES INC, ANALYSIS & TECHNOLOGY INC, BOTTOMLINE TECHNOLOGIES INC, IMAGE GUIDED TECHNOLOGIES INC, CREATIVE TECHNOLOGY LTD, EZGO TECHNOLOGIES LTD, EXCO RESOURCES INC, COMVERSE TECHNOLOGY INC, ADVANCED ANALOGIC TECHNOLOGIES INC]",[],"[0.0000, 0.1882, 0.1944, 0.1957, 0.2052, 0.2220, 0.2248, 0.2397, 0.2410, 0.2488, 0.2500, 0.2539, 0.2544, 0.2546, 0.2546, 0.2559, 0.2560, 0.2605, 0.2608, 0.2620]"


### Normalized Name
In NASDAQ, the company official is different!

- Incorrect: `Aspect Development`
- Correct (Official): `ASPECT DEVELOPMENT INC`